In [1]:
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler
import torch
import os
import math
import re
from itertools import groupby
from PIL import Image

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append('../')

from lora_diffusion import evaluate_pipe, prepare_clip_model_sets, LoraInjectedConv2d, LoraInjectedLinear, patch_pipe, tune_lora_scale, parse_safeloras
from lora_diffusion.lora import _find_modules, UNET_CROSSATTN_TARGET_REPLACE, DEFAULT_TARGET_REPLACE
from reg_lora.visual import visualize_images
import safetensors
import time

os.environ["DISABLE_TELEMETRY"] = 'YES'
os.environ["HTTP_PROXY"] = "http://localhost:8890"
os.environ["HTTPS_PROXY"] = "http://localhost:8890"
model_id = "runwayml/stable-diffusion-v1-5"
device = torch.device("cuda:6")
time.sleep(0.01)
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16,local_ckpt_files_only=True,revision='39593d5650112b4cc580433f6b0435385882d819').to(device)
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
torch.manual_seed(0)

import glob
from collections import defaultdict
preped_clip = prepare_clip_model_sets()
instance_data_dir = '/data/zhicai/code/Text-regularized-customization/dataset/data/dog'
class_tokens = 'dog'
custom_token = '<krk1> dog'

lora_ckpts = sorted(glob.glob("../outputs/checkpoints/dog/ewc_reg_shareTI/baseline_lora_rank_10/lora_weight_s*.safetensors"))
logs = defaultdict(list)
for lora_ckpt in lora_ckpts:
    print(lora_ckpt)
    patch_pipe(
        pipe,
        lora_ckpt,
        patch_text=False,
        patch_ti=True,
        patch_unet=True,
        filter_crossattn_str = 'cross+self'
    )


    test_image_path = instance_data_dir
    images = []
    for file in os.listdir(test_image_path):
        if (
            file.lower().endswith(".png")
            or file.lower().endswith(".jpg")
            or file.lower().endswith(".jpeg")
        ):
            images.append(
                Image.open(os.path.join(test_image_path, file))
            )

    log = evaluate_pipe(
        pipe,
        target_images=images,
        class_token=class_tokens,
        learnt_token=custom_token,
        n_test=10,
        n_step=50,
        clip_model_sets=preped_clip,
    ),
    logs['text_alignment_avg'].append(log["text_alignment_avg"])
    logs['image_alignment_avg'].append(log["image_alignment_avg"])


2023-08-21 17:04:03.717772: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Keyword arguments {'local_ckpt_files_only': True} are not expected by StableDiffusionPipeline and will be ignored.


In [19]:
import glob
from collections import defaultdict
instance_data_dir = '/data/zhicai/code/Text-regularized-customization/dataset/data/dog'
class_tokens = 'dog'
custom_token = '<krk1> dog'

lora_ckpts = sorted(glob.glob("../outputs/checkpoints/dog/ewc_reg_shareTI/ewc_gen_weight_0.01_rank_10_seed_0/lora_weight_s*.safetensors"))
logs = defaultdict(list)
for lora_ckpt in lora_ckpts:
    print(lora_ckpt)
    patch_pipe(
        pipe,
        lora_ckpt,
        patch_text=False,
        patch_ti=True,
        patch_unet=True,
        filter_crossattn_str = 'cross+self'
    )


    test_image_path = instance_data_dir
    images = []
    for file in os.listdir(test_image_path):
        if (
            file.lower().endswith(".png")
            or file.lower().endswith(".jpg")
            or file.lower().endswith(".jpeg")
        ):
            images.append(
                Image.open(os.path.join(test_image_path, file))
            )

    log = evaluate_pipe(
        pipe,
        target_images=images,
        class_token=class_tokens,
        learnt_token=custom_token,
        n_test=10,
        n_step=50,
        clip_model_sets=preped_clip,
    ),
    logs['text_alignment_avg'].append(log[0]["text_alignment_avg"])
    logs['image_alignment_avg'].append(log[0]["image_alignment_avg"])
    print(log[0]["text_alignment_avg"])


../outputs/checkpoints/dog/ewc_reg_shareTI/ewc_gen_weight_0.01_rank_10_seed_0/lora_weight_s1100.safetensors
<krk1>
The tokenizer already contains the token <krk1>.
Replacing <krk1> embedding.


  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

0.24952781200408936


In [20]:
logs

defaultdict(list,
            {'text_alignment_avg': [0.24165284633636475,
              0.2540029287338257,
              0.24662339687347412,
              0.2507498860359192,
              0.24330782890319824,
              0.24922648072242737,
              0.23818202316761017,
              0.2512438893318176,
              0.23951737582683563,
              0.24952781200408936],
             'image_alignment_avg': [0.6247295141220093,
              0.6468112468719482,
              0.6452803015708923,
              0.6580638289451599,
              0.6600483655929565,
              0.6572619676589966,
              0.6719450950622559,
              0.672776460647583,
              0.6429901719093323,
              0.6558781862258911]})

In [21]:
logs

defaultdict(list,
            {'text_alignment_avg': [0.24165284633636475,
              0.2540029287338257,
              0.24662339687347412,
              0.2507498860359192,
              0.24330782890319824,
              0.24922648072242737,
              0.23818202316761017,
              0.2512438893318176,
              0.23951737582683563,
              0.24952781200408936],
             'image_alignment_avg': [0.6247295141220093,
              0.6468112468719482,
              0.6452803015708923,
              0.6580638289451599,
              0.6600483655929565,
              0.6572619676589966,
              0.6719450950622559,
              0.672776460647583,
              0.6429901719093323,
              0.6558781862258911]})